# Robot Painter

**Learning Objectives:**
- Understand how to construct a system that connects trajectory sources, controllers, and the robot station to command an arm.
- Understand how to define a trajectory with key frames and construct the trajectory from the key frames.

**What You'll Implement:**
1. A custom `IIWA_Painter` class that uses a `PoseTrajectorySource` connected to a DiffIK controller.
2. Defining the key frames of a circular trajectory and constructing a trajectory from the keyframes
---


## Setup and Imports

Let us first import our standard drake functionality

In [ ]:
import numpy as np
from pydrake.all import (
    AbstractValue,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    LeafSystem,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_robot_painter import TestRobotPainter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation

## Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!


In [ ]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

## Part 1: Create the Painter

We will start by creating a custom `LeafSystem` that outputs a `RigidTransform` at each time step. Then, we will create the `IIWA_Painter` class that uses a `PoseTrajectorySource` connected to a DiffIK controller to command the robot arm.

**YOUR TASK**: We've provided the scaffolded code. Your job is to fill in the TODOs. You should read through the parts that are already implemented to understand how the system is working.

In [ ]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory: PiecewisePose) -> None:
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory

        # Used when the output is a Python object, not a numpy array
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context: Context, output: AbstractValue) -> None:
        # Gets the pose specified by the trajectory for the current time
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))


class IIWA_Painter:
    def __init__(self, traj: PiecewisePose | None = None) -> None:
        builder = DiagramBuilder()
        scenario_data = """
        directives:
        - add_directives:
            file: package://manipulation/clutter.dmd.yaml
        model_drivers:
            iiwa: !IiwaDriver
                control_mode: position_only
                hand_model_name: wsg
            wsg: !SchunkWsgDriver {}
        """
        scenario = LoadScenario(data=scenario_data)
        self.station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
        self.plant = self.station.GetSubsystemByName("plant")

        # Remove joint limits from the wrist joint.
        self.plant.GetJointByName("iiwa_joint_7").set_position_limits(
            [-np.inf], [np.inf]
        )
        # This gives a MultibodyPlant that the station exposes specifically for controllers. It’s a lightweight, kinematics-only model of the IIWA (no contacts, no geometry sim).
        controller_plant = self.station.GetSubsystemByName(
            "iiwa_controller_plant_pointer_system",
        ).get()

        # optionally add trajectory source
        if traj is not None:
            # Create a PoseTrajectorySource and add it to the diagram
            traj_source = builder.AddSystem(PoseTrajectorySource(traj))

            # Add a Differential IK controller to the diagram
            self.controller = AddIiwaDifferentialIK(
                builder,
                controller_plant,
                frame=controller_plant.GetFrameByName("body"),
            )

            # Connect the output port of the pose trajectory to the input port of the controller
            builder.Connect(
                traj_source.get_output_port(),
                self.controller.get_input_port(0),
            )

            # Connect the output port of the controller to the iiwa position input port of the station
            builder.Connect(
                self.controller.get_output_port(),
                self.station.GetInputPort("iiwa.position"),
            )

            # Here, we connect the iiwa state estimated port to the controller's robot state port
            builder.Connect(
                self.station.GetOutputPort("iiwa.state_estimated"),
                self.controller.GetInputPort("robot_state"),
            )
        else:
            iiwa_position = builder.AddSystem(ConstantVectorSource(np.zeros(7)))
            builder.Connect(
                iiwa_position.get_output_port(),
                self.station.GetInputPort("iiwa.position"),
            )

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(
            wsg_position.get_output_port(),
            self.station.GetInputPort("wsg.position"),
        )

        self.diagram = builder.Build()

        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext()
        self.diagram.ForcedPublish(context)

    def visualize_frame(
        self,
        name: str,
        X_WF: RigidTransform,
        length: float = 0.15,
        radius: float = 0.006,
    ) -> None:
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

    def CreateDefaultContext(self) -> Context:
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, context)

        # provide initial states
        q0 = np.array(
            [
                1.40666193e-05,
                1.56461165e-01,
                -3.82761069e-05,
                -1.32296976e00,
                -6.29097287e-06,
                1.61181157e00,
                -2.66900985e-05,
            ]
        )
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])

        return context

    def get_X_WG(self, context: Context | None = None) -> RigidTransform:
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG

    def paint(self, sim_duration: float = 20.0) -> None:
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)

        meshcat.StartRecording(set_visualizations_while_recording=False)
        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        meshcat.PublishRecording()

# Part 2: Define and Construct the Circular Trajectory
In the lecture, we learned the basics of spatial transformations. In this exercise, you will design a circular planar trajectory like the one below for the iiwa arm to follow, like a robot painting in the air! 

**YOUR TASK**
1. Compute the key frames of the circular trajectory.
2. Construct interpolated trajectory from the key frames

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/robot_painter_circle.png" width="700">

The x and y axis in the diagram above are from the world frame.

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/robot_painter_screenshot.png" width="700">

## Define Key Frames

The screenshot above visualizes the key frames of the circular trajectory. The key frames illustrate the poses of the gripper in the world frame at different time steps along the trajectory. First, you should notice from the visualization above that the gripper frame is different from the world frame. In particular, the +y axis of the gripper frame points vertically downward, and the +z axis of the gripper points backward. This is an important observation for this exercise.

**YOURK TASK**: In the `compose_circular_key_frames` method below, you will compute the key frames of the circular trajectory. For this exercise, we would like to have our arm rotate counterclockwise about the +z axis in the world frame. Besides, we would like the +z axis of the gripper frame to always point toward the center of the circle.

Note: you can compose arbitrary rotations via `MakeXRotation`, `MakeYRotation`, and `MakeZRotation` methods. Their names are quite self-explanatory. There is an example below

In [ ]:
RotationMatrix.MakeYRotation(np.pi / 6.0)

In [ ]:
def compose_circular_key_frames(
    thetas: list[float], X_WCenter: RigidTransform, radius: float
) -> list[RigidTransform]:
    """
    Construct a set of keyframe poses arranged evenly on a circle around a
    reference "center" pose.

    Args:
        thetas list[float]:
            List/array of angles in **radians**. Each angle defines one keyframe
            location on the circle. Angles are measured around the +Y axis of
            the center frame.
        X_WCenter (RigidTransform):
            Pose of the "circle center" frame expressed in the world frame W.
            This sets both the *location* of the circle in space and the *axes*
            that define its plane.
        radius (float):
            Distance from the circle center to each keyframe, in meters.

    Returns:
        list[RigidTransform]:
            A list of end-effector poses in the world frame, that define the circular trajectory.
    """

    key_frame_poses_in_world = []
    for theta in thetas:
        # TODO: Get a series of poses that defines the circular trajectory
        #       The poses should be in the world frame
        pass

    return key_frame_poses_in_world

We define the rigid transform of the center of the circle as well as the radius of the circle below. For this exercise, we are using 10 keyframes, and we define the theta values accordingly.

In [ ]:
# Define the radius of the circle and the center of the circle in the world frame
radius = 0.1
p0 = [0.45, 0.0, 0.4]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WCenter = RigidTransform(R0, p0)

# Define the number of keyframes and thetas accordingly
num_key_frames = 10
thetas = np.linspace(0, 2 * np.pi, num_key_frames)

You may find `visualize_frame` method helpful to visualize rigid transforms. The cell below creates a painter object, computes the rigid transform of the current gripper pose, and draws a frame of that pose in meshcat. Note that the frame drawn here is not attached to any body in the scene. They are for visualization only.

In [ ]:
painter = IIWA_Painter()
X_WG = painter.get_X_WG()
painter.visualize_frame("gripper_current", X_WG)

We can also visualize the poses of the key frames of our circular trajectory. The cell below computes the key frames and visualizes them in meshcat. Make sure your poses match the poses in the screenshot above.

In [ ]:
def visualize_key_frames(frame_poses: list[RigidTransform]):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame("frame_{}".format(i), pose, length=0.05)


key_frame_poses = compose_circular_key_frames(thetas, X_WCenter, radius)
visualize_key_frames(key_frame_poses)

## Construct Trajectory

Now that we have the key frames of our trajectory, we can construct the trajectory to interpolate the positions and orientations of key frames using `PiecewisePose`.

In [ ]:
# Get the initial pose of the gripper and get the keyframes for the trajectory
X_WGinit = painter.get_X_WG()
key_frame_poses = [X_WGinit] + compose_circular_key_frames(thetas, X_WCenter, radius)

# Here we define the total time of our simulation
total_time = 20
times = np.linspace(0, total_time, num_key_frames + 1)

# Use PiecewisePose to construct the trajectory
traj = PiecewisePose.MakeLinear(times, key_frame_poses)

Now you should be able to visualize the execution of the circular painting. Use it to confirm that the gripper moves counterclockwise following the key frames previously drawn in the scene.

In [ ]:
painter = IIWA_Painter(traj)
painter.paint(sim_duration=total_time)

When you are done, you can also use the following code to check your implementation.

In [ ]:
Grader.grade_output([TestRobotPainter], [locals()], "results.json")
Grader.print_test_results("results.json")

# VERIFICATION IN GRADESCOPE 

**Prerequisites:** You must complete ALL the TODOs above before these verification exercises will work!

**Instructions:** Implement the exercises below and upload your solutions to Gradescope

## Verification 1: Keyframe Position

**Question:** Using the default configurations above for the circular trajectory, what is the position of the last keyframe? Give your answer to 4 decimal places.

In [ ]:
# TODO: Implement the verification exercise

## Verification 2: Video

**Task:** Play the simulation of your robot following the circular trajectory and upload a screen recording of it to Gradescope (remember to make sure its an mp4 file with size (much) less than 500 MB).  Gradescope should indicate "Submitted"; the points for this question will be assigned when the staff grades this problem.